<a href="https://colab.research.google.com/github/Nael5/PDP_ETUDIANTS_01_TD06_GP04/blob/master/QwenVL_sur_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install "transformers>=4.46.0" accelerate qwen-vl-utils pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 22.1 MB/s eta 0:00:00


In [3]:
from PIL import Image
import io, requests, torch
from transformers import AutoTokenizer, AutoProcessor, Qwen2VLForConditionalGeneration
from qwen_vl_utils import process_vision_info

# --- Vérif GPU ---
assert torch.cuda.is_available(), "Pas de GPU CUDA détecté. Dans Colab: Runtime > Change runtime type > T4 GPU."
print("GPU:", torch.cuda.get_device_name(0))

# --- Modèle ---
model_name = "Qwen/Qwen2-VL-2B-Instruct"  # petit & rapide ; passe à 7B si besoin de qualité ↑
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,   # T4 -> float16
    device_map="auto",
).eval()

# Astuce: si tu vois un message "Flash attention not available", décommente la ligne suivante:
# torch.backends.cuda.enable_flash_sdp(False)

# --- Utilitaires ---

def load_image_from_path(path: str) -> Image.Image:
    img = Image.open(path).convert("RGB")
    return img

def load_image_from_url(url: str) -> Image.Image:
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    img = Image.open(io.BytesIO(resp.content)).convert("RGB")
    return img

# Choisis l’une des deux lignes ci-dessous:
image = load_image_from_path("/content/monschema.png")
# image = load_image_from_url("https://picsum.photos/1024")



# Redimensionner légèrement si VRAM serrée (optionnel, le processor peut gérer tout seul)
# 672 fonctionne bien sur T4 pour 2B; augmente si tu veux un peu plus de détail
image.thumbnail((672, 672))

query = "Décris l'image en français."

messages = [
    {"role": "user", "content": [
        {"type": "image", "image": image},
        {"type": "text", "text": query}
    ]}
]

# Prétraitement
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt"
).to(model.device)

# Génération
with torch.no_grad():
    out_ids = model.generate(
        **inputs,
        max_new_tokens=160,   # augmente/réduis selon ton besoin
        do_sample=True,
        temperature=0.2,      # 0.2–0.7: plus créatif => plus grand
        top_p=0.9
    )

print(processor.batch_decode(out_ids, skip_special_tokens=True)[0])


GPU: Tesla T4


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

system
You are a helpful assistant.
user
Décris l'image en français.
assistant
L'image montre un plan sphérique bleu avec deux points A et B. Un vecteur d'effort F est dirigé vers le haut et pointe vers le haut. Un vecteur de moment M est dirigé vers le haut et pointe vers le haut. Un vecteur de moment M est dirigé vers le bas et pointe vers le bas. Un vecteur de moment M est dirigé vers le bas et pointe vers le bas.
